# What we're going to cover

- Downloading a text dataset
- Visualizing text data
- Converting text into numbers using tokenization
- Turning our tokenized text into an embedding
- Modelling a text dataset
    - Starting with a baseline (TF-IDF)
    - Building several deep learning text models
        - Dense, LSTM, GRU, Conv1D, Transfer learning
- Comparing the performance of each our models
- Combining our models into an ensemble
- Saving and loading a trained model
- Find the most wrong predictions

In [1]:
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")

Notebook last run (end-to-end): 2025-12-16 23:08:13.413418


# Check for GPU

In [2]:
# Check for GPU
!nvidia-smi -L

GPU 0: NVIDIA T500 (UUID: GPU-2e388ea4-5136-87e9-1d14-8e74849e1774)


# Get helper functions

In [4]:
# Download helper functions script
#!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

import urllib.request

url = "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"
urllib.request.urlretrieve(url, "helper_functions.py")


('helper_functions.py', <http.client.HTTPMessage at 0x2538e681c10>)

In [11]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Download a text dataset from Kaggle

In [12]:
# Download data (same as from Kaggle)import urllib.request
url = "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
urllib.request.urlretrieve(url, "nlp_getting_started.zip")

import zipfile
with zipfile.ZipFile("nlp_getting_started.zip", "r") as zip_ref:
    zip_ref.extractall("nlp_getting_started")  # folder where files will be extracted


# Visualizing a text dataset

In [14]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("./nlp_getting_started/train.csv")
test_df = pd.read_csv("./nlp_getting_started/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [15]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [16]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [17]:
# How many examples of each class?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

Since we have two target values, we're dealing with a binary classification problem.

It's fairly balanced too, about 60% negative class (target = 0) and 40% positive class (target = 1).

Where,
- 1 = a real disaster Tweet
- 0 = not a real disaster Tweet

In [18]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


Alright, seems like we've got a decent amount of training and test data. If anything, we've got an abundance of testing examples, usually a split of 90/10 (90% training, 10% testing) or 80/20 is suffice.

In [19]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
@chrisroth98 @chaselabsports in an emergency situation late in the year. Not as a plan in camp

---

Target: 1 (real disaster)
Text:
@onihimedesu the whole city is trapped! You can't leave the city! This was supposed to be a normal sports manga wit a love triangle (c)

---

Target: 0 (not real disaster)
Text:
The good thing is that the #Royals won't face a newbie in the playoffs. No real reason to panic.

---

Target: 1 (real disaster)
Text:
Our Chemical Spill Cleanup videos will prepare you for an emergency situation in the lab. http://t.co/UMQbyRUPBd

---

Target: 0 (not real disaster)
Text:
Yumiko jumped in surprise as the fire shot upwards into the air and exploded caught off guard~ 'woah...' She had--( @LenkaIsWaifu )

---



# Split data into training and validation sets

In [21]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split                                                                (train_df_shuffled["text"].to_numpy(),                                                        train_df_shuffled["target"].to_numpy(),
                                                test_size=0.1, # dedicate 10% of samples to validation set
                                                random_state=42) # random state for reproducibility

In [22]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [23]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

## Text vectorization (tokenization)

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [25]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [33]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [34]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [35]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [36]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@SwellyJetEvo Disneyland! Tacos there are bomb!      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7866,    1, 7835,   74,   22,  108,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [37]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Bottom 5 least common words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


# Creating an Embedding using an Embedding Layer

In [41]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

C:\Users\mdsan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_1, built=False>

In [42]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@UniversityofLaw For the people who died in Human Experiments by Unit 731 of Japanese military http://t.co/vVPLFQv58P http://t.co/eG1fsKqBv6      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02978376,  0.03231963,  0.04895322, ..., -0.01395314,
         -0.01427661, -0.02229604],
        [-0.00250782,  0.02038288, -0.03441892, ...,  0.01141378,
          0.00399121, -0.02175549],
        [ 0.02991045, -0.02818593,  0.03505747, ..., -0.00922094,
         -0.00011979, -0.02742982],
        ...,
        [ 0.03176517, -0.01581382,  0.00212751, ..., -0.04326018,
         -0.02403592,  0.00841372],
        [ 0.04644514, -0.03658871,  0.04242877, ..., -0.01580913,
          0.01060022, -0.01811805],
        [-0.04380089,  0.01523251, -0.03760218, ...,  0.00585424,
          0.0041052 , -0.04775751]]], shape=(1, 15, 128), dtype=float32)>

In [43]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.02978376,  0.03231963,  0.04895322, -0.04285923,  0.03993431,
       -0.02398149,  0.01329106,  0.01308082, -0.031224  ,  0.0203524 ,
       -0.03853511,  0.0085867 ,  0.01396158, -0.03515278,  0.02296497,
       -0.048386  ,  0.01588663, -0.01415107,  0.00369243, -0.01403586,
       -0.01531465,  0.0042456 ,  0.00576194,  0.03868954,  0.03752667,
        0.01208328, -0.03930784,  0.01539869,  0.04482067, -0.04906214,
       -0.02442045,  0.04259814,  0.03053974,  0.04593151,  0.03694005,
       -0.01483405,  0.02810654, -0.0477473 , -0.01746254,  0.00120059,
        0.00157058, -0.03467734, -0.0351581 ,  0.02872572, -0.04904504,
       -0.03028265,  0.01369286, -0.02979715,  0.03430815,  0.01863444,
        0.04671285,  0.03175483, -0.02623777,  0.03272581, -0.04553641,
        0.04228273,  0.04206872,  0.03585744,  0.00557054, -0.04624077,
       -0.00473021,  0.0094259 , -0.01640278, -0.02163906, -0.03275955,
        0.011500